### Notebook sur le jeu de test du Titanic


In [62]:
import os.path
trainDataSetPath = os.path.join("datasets","titanic","train.csv")
testDataSetPath = os.path.join("datasets","titanic","test.csv")
import pandas as pd 

In [90]:

trainDf = pd.read_csv(trainDataSetPath)
trainDf.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [91]:
trainDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [92]:
from titanic.normalize import normalize 
trainDf = normalize.normalize(trainDf)
trainDf.head(10) 

28.0


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Gender
0,0,3,0,22.0,1,0,7.2500,0,0
1,1,1,1,38.0,1,0,71.2833,1,1
2,1,3,1,26.0,0,0,7.9250,0,1
3,1,1,1,35.0,1,0,53.1000,0,1
4,0,3,0,35.0,0,0,8.0500,0,0
5,0,3,0,28.0,0,0,8.4583,2,0
6,0,1,0,54.0,0,0,51.8625,0,0
7,0,3,0,2.0,3,1,21.0750,0,0
8,1,3,1,27.0,0,2,11.1333,0,1
9,1,2,1,14.0,1,0,30.0708,1,1


In [93]:
corr_matrix = trainDf.corr()
corr_matrix['Survived'] .sort_values(ascending = False)

Survived    1.000000
Sex         0.543351
Gender      0.543351
Fare        0.257307
Embarked    0.106811
Parch       0.081629
SibSp      -0.035322
Age        -0.064910
Pclass     -0.338481
Name: Survived, dtype: float64

#### Créer un jeu de test et un jeu d'entrainement 

In [94]:
from sklearn.model_selection import ShuffleSplit 

split = ShuffleSplit(n_splits=1, test_size=0.2,random_state=42) 
for train_index, test_index in split.split(trainDf,trainDf['Fare']):
    trainSet = trainDf.loc[train_index] 
    testSet = trainDf.loc[test_index]

#### On calcule les prédictions

###  On va utiliser un DecisionTreeClassifier 
####  Recherche des meilleurs paramétres par quadrillage

In [100]:
from sklearn.model_selection import GridSearchCV 
from sklearn.tree import DecisionTreeClassifier  
param_grid = [{ 
    'criterion' : [ 'gini', 'entropy', 'log_loss' ] , 
    'splitter'  : [ 'best', 'random' ] , 
    'max_depth' : [2,3,4] , 
    "min_samples_split" : [50 , 25 , 10 ]}
]
dcl = DecisionTreeClassifier()
gridSearch = GridSearchCV(dcl , param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True )
target  = trainSet['Survived'] 
gridSearch.fit(trainSet,target )
gridSearch.best_params_

{'criterion': 'gini',
 'max_depth': 2,
 'min_samples_split': 50,
 'splitter': 'best'}

In [101]:
from sklearn.tree import DecisionTreeClassifier 

dcl = DecisionTreeClassifier(criterion = 'gini',
                             max_depth= 2,
                             min_samples_split = 50,
                             splitter='best') 
dcl.fit(trainSet,target)  

DecisionTreeClassifier(max_depth=2, min_samples_split=50)

### Evaluation des predictions 

In [102]:
observed = testSet["Survived"]
observed = observed.to_numpy(dtype='int64', copy=True) 
predictions = dcl.predict(testSet)
print(observed)
predictions

[1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0 1 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1
 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1]


array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1], dtype=int64)

In [103]:
def evaluateResult(predictions):
    nbErrors = 0 
    for i in range(0,len(predictions)):
        if predictions[i] != observed[i]:
            nbErrors += 1
    ratio = (nbErrors / len(predictions) ) * 100 
    return nbErrors, ratio 

nbErrors , ratio = evaluateResult(predictions)  
print(f' nb Erreurs pour la classe {dcl.__class__.__name__}: {nbErrors} ({ratio:4.2f}%)')    
print()

 nb Erreurs pour la classe DecisionTreeClassifier: 0 (0.00%)



### Test d'un SGDClassifier 

In [104]:
from sklearn.linear_model import SGDClassifier 
sgdClf = SGDClassifier(random_state=42) 
# entraînement 
sgdClf.fit(trainSetClean,target)
predictions = sgdClf.predict(testSet2)
nbErrors , ratio = evaluateResult(predictions)  
print(f' nb Erreurs pour la classe {sgdClf.__class__.__name__}: {nbErrors} ({ratio:4.2f}%)')   

 nb Erreurs pour la classe SGDClassifier: 34 (18.99%)


In [105]:
from sklearn.linear_model import SGDRegressor 
sgdReg = SGDRegressor(max_iter=1000, tol=1e-3,penalty=None,eta0=0.1) 
# entraînement 
sgdReg.fit(trainSetClean,target.ravel())
predictions = sgdReg.predict(testSet2)
nbErrors , ratio = evaluateResult(predictions)  
print(f' nb Erreurs pour la classe {sgdReg.__class__.__name__}: {nbErrors} ({ratio:4.2f}%)')  

 nb Erreurs pour la classe SGDRegressor: 179 (100.00%)


In [106]:
from sklearn.linear_model import ElasticNet 
enReg = ElasticNet(alpha=0.1, l1_ratio=0.5) 
# entraînement 
enReg.fit(trainSetClean,target.ravel())
predictions = enReg.predict(testSet2)
nbErrors , ratio = evaluateResult(predictions)  
print(f' nb Erreurs pour la classe {enReg.__class__.__name__}: {nbErrors} ({ratio:4.2f}%)')  

 nb Erreurs pour la classe ElasticNet: 179 (100.00%)


In [107]:
from sklearn.svm import LinearSVC 
svmLinear = LinearSVC(C=1, loss="hinge",dual="auto",max_iter=10**6) 
# entraînement 
svmLinear.fit(trainSetClean,target)
predictions = svmLinear.predict(testSet2)
nbErrors , ratio = evaluateResult(predictions)  
print(f' nb Erreurs pour la classe {svmLinear.__class__.__name__}: {nbErrors} ({ratio:4.2f}%)') 

 nb Erreurs pour la classe LinearSVC: 39 (21.79%)


In [111]:
from sklearn.model_selection import GridSearchCV
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

@ignore_warnings(category=ConvergenceWarning)
def gridsearchcv(model):

    param_grid = [{ 
        'penalty' : [ 'l1', 'l2' ] , 
        'loss'  : [ 'hinge', 'squared_hinge' ] , 
        'dual' : [True,False] , 
        "tol" : [10 ** -n for n in range(1,4) ] , 
        "C" : [0.5,1,1.5,2],
        "random_state" : [42] , 
        "max_iter" : [ 10 ** 6   ]}
    ]

    gridSearch = GridSearchCV(model , param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True )
    target  = trainSet['Survived'] 
    gridSearch.fit(trainSet,target )
    return gridSearch.best_params_

lSVC = LinearSVC()
bp = gridsearchcv(lSVC)
print(bp) 

{'C': 0.5, 'dual': True, 'loss': 'hinge', 'max_iter': 1000000, 'penalty': 'l2', 'random_state': 42, 'tol': 0.1}


C:\Users\stallard\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
240 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\stallard\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\stallard\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\stallard\AppData\Local\anaconda3\Lib\site-package

### Test de LinearSVC mais en utilisant un StandardScaler sur les données en entrée 

In [88]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()
svmLinear = LinearSVC(C=0.5, dual = True, 
                      loss= 'squared_hinge', 
                       max_iter = 1000000, 
                       penalty = 'l2', 
                       random_state = 42, 
                       tol = 0.01 ) 
# entraînement 
trainSetCleanSc = sc.fit_transform(trainSetClean)
svmLinear.fit(trainSetCleanSc,target)
testSetSc = sc.fit_transform(testSet2)
predictions = svmLinear.predict(testSetSc)
print(observed)
predictions


[1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0 1 0 1 1 1
 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1
 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1]


array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1], dtype=int64)

In [89]:
nbErrors , ratio = evaluateResult(predictions)  
print(f' nb Erreurs pour la classe {svmLinear.__class__.__name__}: {nbErrors} ({ratio:4.2f}%)') 

 nb Erreurs pour la classe LinearSVC: 37 (20.67%)
